### BigQuery Table Definitions for Stock Market Project


In [28]:
#assign dependencies and constants
from google.cloud import bigquery

PROJECT_ID = 'expiry-week'
bq_client = bigquery.Client(project=PROJECT_ID)


In [10]:
def create_dataset(dataset_name):
    """create a dataset in the currently assigned project"""
    dataset_id = "{}.{}".format(PROJECT_ID, dataset_name)
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = 'US'
    dataset = bq_client.create_dataset(dataset, timeout=30)
    print("Dataset {} has been created".format(dataset_id))
    

In [32]:
def create_option_quote_table(underlying_symbol):
    """
    Create a BigQuery table in the option_quotes dataset for the supplied underlying symbol
    """
    schema = """
        CREATE TABLE IF NOT EXISTS expiry-week.option_quotes.{}
        (
            quote_date DATE NOT NULL OPTIONS(description="market closing date for which the data was captured"),  
            expiry_date DATE NOT NULL OPTIONS(description="date on which the options expire"),
            days_to_expiry INT64 NOT NULL OPTIONS(description="calendar days between quote_date and expiry_date not including the quote_date"),
            underlying_price NUMERIC NOT NULL OPTIONS(description="the closing price of the underlying asset"),
            atm_price NUMERIC NOT NULL OPTIONS(description="adjusted underlying price such that the implied volatility of calls an puts are the same"),
            atm_iv NUMERIC NOT NULL OPTIONS(description="the implied volatility for both calls and puts using the atm_price as the current price"),
            strike_prices STRUCT
            <
                strike_price NUMERIC NOT NULL OPTIONS(description="price at which an option can be excercised"),
                call_bid NUMERIC NOT NULL OPTIONS(description="closing bid price for the call option"),
                call_ask NUMERIC NOT NULL OPTIONS(description="closing ask price for the call option"),
                call_volume NUMERIC NOT NULL OPTIONS(description="number of call contracts traded"),
                call_open_interest NUMERIC OPTIONS(description="number of open call contracts"),
                call_moneyness NUMERIC NOT NULL OPTIONS(description="probability of call option closing in the money on expiry date (based on atm_iv)"),
                call_iv NUMERIC NOT NULL OPTIONS(description="implied volatiity of the call option using the midpoint between the call bid and call ask price"),
                put_bid NUMERIC NOT NULL OPTIONS(description="closing bid price for the put option"),
                put_ask NUMERIC NOT NULL OPTIONS(description="closing ask price for the put option"),
                put_volume NUMERIC NOT NULL OPTIONS(description="number of put contracts traded"),
                put_open_interest NUMERIC NOT NULL OPTIONS(description="number of open call contracts"),
                put_moneyness NUMERIC NOT NULL OPTIONS(description="probability of put option closing in the money on expiry date (based on atm_iv)"),
                put_iv NUMERIC NOT NULL OPTIONS(description="implied volatiity of the put option using the midpoint between the put bid and put ask price")
            >,
            sampling_key NUMERIC NOT NULL OPTIONS(description="a random number between 0 and 1. Facilitates repeatable data sampling without the need for a hash key")
        )
        PARTITION BY quote_date
        CLUSTER BY expiry_date
        OPTIONS
            (description="DAILY STOCK OPTION QUOTES")
        """
    job = bq_client.query(schema.format(underlying_symbol))
    print("Table {} has been created in dataset option_quotes".format(underlying_symbol))
    